In [ ]:
import os
import re
from dotenv import load_dotenv
from langchain.text_splitter import CharacterTextSplitter
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import Chroma
from langchain_openai import OpenAIEmbeddings
from bs4 import BeautifulSoup

In [ ]:
#load_dotenv()

False

In [56]:
current_dir = "E:/JupyterProjects/Internship"
db_dir = os.path.join(current_dir, "db")
persistent_directory = os.path.join(db_dir, "db_captechu")

In [86]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
import os

filepaths = [
    r"E:\JupyterProjects\Internship\2024-25 Clubs and Orgs Guide Printable.txt",
    r"E:\JupyterProjects\Internship\student_guide_to_the_myCapitol_updated_172020.pdf",
    r"E:\JupyterProjects\Internship\2024-25 Clubs and Orgs Guide Printable.txt",
    r"E:\JupyterProjects\Internship\Course Catalog 2022-2023.txt"
]

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1200,  # Adjust
    chunk_overlap=100  # Adjust
)

# directory
os.makedirs("doc_scraped_data", exist_ok=True)

# Process each file path
for filepath in filepaths:
    try:
        # Readcurrent .txt file
        with open(filepath, 'r', encoding='utf-8') as f:
            content = f.read()
        
        # Split into chunks
        para_list = text_splitter.create_documents(texts=[content])

        # Save each chunk into a separate text file
        for i, doc in enumerate(para_list):
            chunk_filename = f"doc_scraped_data/{os.path.basename(filepath)}_chunk_{i}.txt"
            
            with open(chunk_filename, "w", encoding="utf-8") as f:
                f.write(doc.page_content)
        
        print(f"Processed and saved chunks for: {filepath}")
    
    except Exception as e:
        print(f"Error processing {filepath}: {e}")

print("All files processed successfully.")


Processed and saved chunks for: E:\JupyterProjects\Internship\2024-25 Clubs and Orgs Guide Printable.txt
Error processing E:\JupyterProjects\Internship\student_guide_to_the_myCapitol_updated_172020.pdf: [Errno 2] No such file or directory: 'E:\\JupyterProjects\\Internship\\student_guide_to_the_myCapitol_updated_172020.pdf'
Processed and saved chunks for: E:\JupyterProjects\Internship\2024-25 Clubs and Orgs Guide Printable.txt
Processed and saved chunks for: E:\JupyterProjects\Internship\Course Catalog 2022-2023.txt
All files processed successfully.


In [ ]:
urls = [
    "https://mycapitol.captechu.edu/ICS/", 
    "https://mycapitol.captechu.edu/ICS/Admissions/", 
    "https://mycapitol.captechu.edu/ICS/Residence_Life/",
    "https://mycapitol.captechu.edu/ICS/Campus_Life/", 
    "https://mycapitol.captechu.edu/ICS/College_Offices/",
    "https://mycapitol.captechu.edu/ICS/Library/", 
    "https://mycapitol.captechu.edu/ICS/Public_Information/",
    "https://mycapitol.captechu.edu/ICS/College_Offices/Advising/", 
    "https://mycapitol.captechu.edu/ICS/College_Offices/Business_Office/",
    "https://mycapitol.captechu.edu/ICS/College_Offices/Financial_Aid_Office/", 
    "https://mycapitol.captechu.edu/ICS/College_Offices/Information_Technology/",
    "https://mycapitol.captechu.edu/ICS/College_Offices/The_Office_of_Registration_and_Records/",
    "https://www.captechu.edu/degrees-and-programs/bachelors-degrees/data-science-bs",
    "https://www.captechu.edu/about-capitol/why-capitol-tech",
    "https://www.captechu.edu/about-capitol/mission-vision-and-goals",
    "https://www.captechu.edu/student-experience/campus-life",
    "https://www.captechu.edu/about-capitol/diversity-equity-and-inclusion",
    "https://www.captechu.edu/professional-success/career-services"
]


documents = []

# Iterate over each URL
for url in urls:
    try:
        #loader for the current URL
        loader = WebBaseLoader([url])
        
        # Load documents
        doc = loader.load()
        
        # Append the documents
        documents.extend(doc)
        
        # Debugging
        print(f"Loaded {len(doc)} documents from {url}")
        
    except Exception as e:
        print(f"Failed to load content from {url}. Error: {e}")
        
def clean_text(raw_text):
    # Remove extra line breaks and unnecessary white spaces
    cleaned_text = re.sub(r'\n+', '\n', raw_text) 
    cleaned_text = re.sub(r'\s{2,}', ' ', cleaned_text) 
    cleaned_text = cleaned_text.strip()
    return cleaned_text

#cleaning and splitting the text
cleaned_texts = []
for doc in documents:
    #Parse and clean the HTML content
    soup = BeautifulSoup(doc.page_content, 'html.parser')
    
    # unwanted sections (e.g., navigation, footer)
    for tag in soup.find_all(['nav', 'footer', 'header']):
        tag.decompose()  # Removes these tags from the soup
    
    # text content
    cleaned_text = soup.get_text(separator="\n\n")  # preserve paragraphs
    cleaned_texts.append(cleaned_text)

# Split text
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1200, chunk_overlap=100)
website_para_list = text_splitter.create_documents(texts=cleaned_texts)

# debugging
print(f"Total number of chunks generated: {len(website_para_list)}")

# directory
os.makedirs("scraped_data", exist_ok=True)

# Save chunks
for i, chunk in enumerate(website_para_list):
    with open(f"scraped_data/chunk_{i}.txt", "w", encoding="utf-8") as f:
        f.write(chunk.page_content)

print(f"Total chunks saved: {len(website_para_list)}")


Loaded 1 documents from https://mycapitol.captechu.edu/ICS/
Loaded 1 documents from https://mycapitol.captechu.edu/ICS/Admissions/
Loaded 1 documents from https://mycapitol.captechu.edu/ICS/Residence_Life/
Loaded 1 documents from https://mycapitol.captechu.edu/ICS/Campus_Life/
Loaded 1 documents from https://mycapitol.captechu.edu/ICS/College_Offices/
Loaded 1 documents from https://mycapitol.captechu.edu/ICS/Library/
Loaded 1 documents from https://mycapitol.captechu.edu/ICS/Public_Information/
Loaded 1 documents from https://mycapitol.captechu.edu/ICS/College_Offices/Advising/
Loaded 1 documents from https://mycapitol.captechu.edu/ICS/College_Offices/Business_Office/
Loaded 1 documents from https://mycapitol.captechu.edu/ICS/College_Offices/Financial_Aid_Office/
Loaded 1 documents from https://mycapitol.captechu.edu/ICS/College_Offices/Information_Technology/
Loaded 1 documents from https://mycapitol.captechu.edu/ICS/College_Offices/The_Office_of_Registration_and_Records/
Loaded 1 do